Прообраз: https://github.com/bigartm/bigartm-book/blob/master/ARTM_example_RU.ipynb

In [39]:
%matplotlib inline
import glob
import os
import matplotlib.pyplot as plt

import artm

In [40]:
BATCH_ADRESS = 'batches_news'

In [41]:
batch_vectorizer = artm.BatchVectorizer(
    data_path=BATCH_ADRESS, data_format='batches')

In [42]:
dictionary = artm.Dictionary()

model_plsa = artm.ARTM(topic_names=['topic_{}'.format(i) for i in range(15)],
                       scores=[artm.PerplexityScore(name='PerplexityScore',
                                                    #use_unigram_document_model=False,
                                                    dictionary=dictionary)],
                       cache_theta=True)

model_artm = artm.ARTM(topic_names=['topic_{}'.format(i) for i in range(15)],
                       scores=[artm.PerplexityScore(name='PerplexityScore',
                                                    #use_unigram_document_model=False,
                                                    dictionary=dictionary)],
                       regularizers=[artm.SmoothSparseThetaRegularizer(name='SparseTheta', tau=-0.15)],
                       cache_theta=True)

In [43]:
if not os.path.isfile(BATCH_ADRESS + '/dictionary.dict'):
    dictionary.gather(data_path=batch_vectorizer.data_path)
    dictionary.save(dictionary_path=BATCH_ADRESS + '/dictionary.dict')

dictionary.load(dictionary_path=BATCH_ADRESS + '/dictionary.dict')
dictionary.load(dictionary_path=BATCH_ADRESS + '/dictionary.dict')

In [44]:
model_plsa.initialize(dictionary=dictionary)
model_artm.initialize(dictionary=dictionary)

In [45]:
model_plsa.scores.add(artm.SparsityPhiScore(name='SparsityPhiScore'))
model_plsa.scores.add(artm.SparsityThetaScore(name='SparsityThetaScore'))
model_plsa.scores.add(artm.TopicKernelScore(name='TopicKernelScore', probability_mass_threshold=0.3))

model_artm.scores.add(artm.SparsityPhiScore(name='SparsityPhiScore'))
model_artm.scores.add(artm.SparsityThetaScore(name='SparsityThetaScore'))
model_artm.scores.add(artm.TopicKernelScore(name='TopicKernelScore', probability_mass_threshold=0.3))

In [46]:
model_artm.regularizers.add(artm.SmoothSparsePhiRegularizer(name='SparsePhi', tau=-0.1))
model_artm.regularizers.add(artm.DecorrelatorPhiRegularizer(name='DecorrelatorPhi', tau=1.5e+5))

In [47]:
model_plsa.num_document_passes = 1
model_artm.num_document_passes = 1

In [48]:
model_plsa.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=15)
model_artm.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=15)

Exception ignored in: <generator object tqdm_notebook.__iter__ at 0x10f1ea3b8>
Traceback (most recent call last):
  File "/Users/kirkira/Code/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm_notebook.py", line 228, in __iter__
    self.sp(bar_style='danger')
AttributeError: 'tqdm_notebook' object has no attribute 'sp'


InvalidOperationException: TopicKernelScoreConfig.class_id @default_class does not exists in n_wt matrix